# Access Text Analysis


In [9]:
import os
import string
import json
import nltk
import glob
import pandas as pd
import matplotlib.pyplot as plt


from shutil import unpack_archive

from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
# Load CSV Files

df_list = []

for f in os.listdir(os.curdir):
    if f.endswith('.csv'):
        print(f)
        df = pd.read_csv(f)
        df_list.append(df)
        

2008.csv
2009.csv
2019.csv
2018.csv
2001.csv
2015.csv
2014.csv
2000.csv
2016.csv
2002.csv
2003.csv
2017.csv
2013.csv
2007.csv
2006.csv
2012.csv
2004.csv
2010.csv
2011.csv
2005.csv
1999.csv
1998.csv


In [16]:
df_list[1]



,year,date,time,speakers,title,abstract,type,tags,slides,video,audio
0,2009,2009-10-01,8:30-9:30,Cory Doctorow (Boing Boing),Copyright vs Universal Access to All Human Kno...,The Internet promises the realization of two o...,keynote,NaN,NaN,NaN,NaN
1,2009,2009-10-01,9:30-10:15,"Richard Akerman (National Research Council, Ot...",Will We Command Our Data?,This presentation provides an overview of rece...,"talk, binkley",NaN,NaN,NaN,NaN
2,2009,2009-10-01,10:45-11:05,"Mark Jordan (Simon Fraser University, Burnaby,...",COPPUL's LOCKSS Private Network,COPPUL (the Council of Prairie and Pacific Uni...,talk,NaN,NaN,NaN,NaN
3,2009,2009-10-01,11:05-11:30,"Brian Owen (Simon Fraser University, Burnaby, BC)",Software Lifecycles & Sustainability: a PKP an...,The PKP and reSearcher open source software su...,talk,NaN,NaN,NaN,NaN
4,2009,2009-10-01,1:00-1:45,"Dan Chudnov (Library of Congress, Washington, DC)",Chudnovian Stuff,The Repository Development Group at the Librar...,talk,NaN,NaN,NaN,NaN
5,2009,2009-10-01,1:45-2:30,"Donald Moses (UPEI, Charlottetown, PEI), Paul ...",IslandLives: A Flexible Digital Repository Fra...,In 2008 the Robertson Library received a priva...,talk,NaN,NaN,NaN,NaN
6,2009,2009-10-01,2:30-3:15,"Stevan Harnad (UQAM, Montreal)",Grasping What is Already Within Immediate Reac...,Open Access (OA) means free online access to p...,talk,NaN,NaN,NaN,NaN
7,2009,2009-10-01,3:45-4:07,Roy Tennant (OCLC),ILS in the Sky with Diamonds,"Cloud computing has become all the rage, but w...",talk,NaN,NaN,NaN,NaN
8,2009,2009-10-01,4:08-4:30,Mike Rylander (Equinox Software),OS ILS,NaN,talk,NaN,NaN,NaN,NaN
9,2009,2009-10-02,8:30-9:15,Peter Rukavina (Reinvented Inc.),Applications vs. Capabilities,(Originally titled Infinite Malleability: Port...,talk,NaN,NaN,NaN,NaN


In [18]:
for df in df_list:
    print(df.abstract)

0                                                   NaN
1     Open source software enjoys the rapt attention...
2     This presentation describes and illustrates th...
3                                                   NaN
4     In 2008 the AlouetteCanada/OurOntario.ca searc...
5     The University of Michigan libraries launched ...
6     Mass digitization is underway at many librarie...
7     Just because a library catalogue provides Web ...
8     A panel presentation about using the open-sour...
9                                                   NaN
10                                                  NaN
11    Libraries today are presented with the challen...
12    This presentation describes the origin, presen...
13    OCLC’s WorldCat Grid offers a wide array of bi...
14    FADIS (Federated Academic Digital Imaging Syst...
15                                                  NaN
16    The current state of the ILS market is forcing...
17    Contemporary libraries need state-of-the-a

In [20]:
#Binary Sentiment
#Looks at all of our tweets and determines which are 'positive', 'negative', or 'neutral' based on binary classification


sid = SentimentIntensityAnalyzer()
summary = {"positive":0,"neutral":0,"negative":0}

for df in df_list:
    for x in df.abstract:
        ss = sid.polarity_scores(x)
        if ss["compound"] == 0.0:
            analysis = analysis.append({"Sent": "NEU", "Text": x},ignore_index=True)
            summary["neutral"] +=1
        elif ss["compound"] > 0.0:
            analysis = analysis.append({"Sent": "POS", "Text": x},ignore_index=True)
            summary["positive"] +=1
        else:
            analysis = analysis.append({"Sent": "NEG", "Text": x},ignore_index=True)
            summary["negative"] +=1

        
print(summary)
print("Done!")

AttributeError: 'float' object has no attribute 'encode'